In [73]:
"""
B-tree

a balanced search tree with more than one value per node

Properties
1. Every node has n keys stored in non-decreasing order; it is
either a leaf node or an internal node;
2. Each internal node contains n+1 pointers to its children; while
leaf nodes have no children;
3. Keys of a node separate the ranges of keys in each subtree;
4. All leafs have the same height;
5. Node has a minimum degree t; Every node other than root must 
contains at least t-1 at most 2t-1 keys.

"""

class Node():
    def __init__(self,t):
        self._children = [None] * (2*t)
        self._key = [None] * (2*t - 1)
        self._leaf = True
        self._t = t
   
    @property
    def children(self):
        return self._children
    
    @property
    def key(self):
        return self._key
    
    @property
    def leaf(self):
        return self._leaf
    
    @property
    def t(self):
        return self._t
    
    @property
    def full(self):
        return self._key[2*self._t-2] != None

    
    @leaf.setter
    def leaf(self,l):
        self._leaf = l
        
    @key.setter
    def key(self,v):
        self._key = v
        
    @children.setter
    def children(self,c):
        self._children = c
        
    def __str__(self, level=0, offset=0, prefix=""):
        return prefix + "     "*level + " ".join([repr(i) for i in self._key])
        
class BTree():
    def __init__(self,t):
        self._root = Node(t)
        self._t = t
    
    
    # print all nodes 
    def display(self):
        print_method = lambda n,l,o,p: print(n.__str__(l,o,p))
        return self._inorder_walk(self._root, 0, 0, "    ", print_method)
    
    # walk all nodes and apply method
    def _inorder_walk(self, node, level, offset, prefix, m):
        if isinstance(node, Node):
            m(node, level, offset, prefix)
            for c in node.children:
                if c != None:
                    m(c, level+1, offset, prefix) 

    # search
    def search(self, value):
        self._search_node(self._root, value)
        
        
    def _search_node(self, x, value):
        for i in range(len(x.key)):
            v = x.key[i]
            if value == v:
                return (x,i)
            elif value < v:
                return self._search_node(x.children[i],value)
        return None
    
        
    def insert(self, k):
        print('insert {0}'.format(k))
        r = self._root
        if r.full:
            print('root full')
            s = Node(self._t)
            self._root = s
            s.leaf = False
            s.children[0] = r
            self._split_child(s,0)
            self._insert_non_full(s,k)
        else:
            print('root not full')
            self._insert_non_full(r,k)
    
    
    # split x's ith child(full node) to two equal halfs
    def _split_child(self, x, i):
        y = x.children[i]
        if y == None:
            return  
        
        t = self._t
        z = Node(t)
        z.leaf = y.leaf
        
        # copy y's key to z
        for j in range(0,t-1):
            z.key[j] = y.key[j+t]
            y.key[j+t] = None
        
        # copy y's pointer to z
        if not y.leaf:
            for j in range(0,t-1):
                z.children[j] = y.children[j+t]
                y.children[j+t] = None                   
        
        # shift x's children one step to the right
        for j in range(len(x.children)-2, i):
            x.children[j+1] = x.children[j]
        x.children[i+1] = z
        
        # shift x's key one step to the right
        for j in range(len(x.key)-1, i):
            x.key[j+1] = x.key[j]
        x.key[i] = y.key[t-1]
        y.key[t-1] = None
        print('done split')
        self.display()

    
    def _insert_non_full(self, x, k):
        print('insert_non_full')
        print(x)
        print(k)
        i = 2*x.t - 3
        if x.leaf:         
            while i >= 0 and (x.key[i] == None or x.key[i] > k):
                x.key[i+1] = x.key[i]
                i = i-1
            print('leaf {0}'.format(i))
            x.key[i+1] = k
        else:
            while i >= 0 and (x.key[i] == None or x.key[i] > k):
                i = i-1
            i = i + 1
            print('children {0}'.format(i))
            if x.children[i].full:
                self._split_child(x,i)
            if x.key[i] != None and k > x.key[i]:
                self._insert_non_full(x.children[i+1], k)
            else:
                self._insert_non_full(x.children[i], k)
            
    
    def delete(self, k):
        pass

In [74]:
bt = BTree(2)

for i in [1,7,4,9,2,3,6,8,5,10,20,32,14,19,27,28,39]:
    bt.insert(i)
    bt.display()
    print('\n')

insert 1
root not full
insert_non_full
None None None
1
leaf -1
    1 None None


insert 7
root not full
insert_non_full
1 None None
7
leaf 0
    1 7 None


insert 4
root not full
insert_non_full
1 7 None
4
leaf 0
    1 4 7


insert 9
root full
done split
    4 None None
         1 None None
         7 None None
insert_non_full
4 None None
9
children 1
insert_non_full
7 None None
9
leaf 0
    4 None None
         1 None None
         7 9 None


insert 2
root not full
insert_non_full
4 None None
2
children 0
insert_non_full
1 None None
2
leaf 0
    4 None None
         1 2 None
         7 9 None


insert 3
root not full
insert_non_full
4 None None
3
children 0
insert_non_full
1 2 None
3
leaf 1
    4 None None
         1 2 3
         7 9 None


insert 6
root not full
insert_non_full
4 None None
6
children 1
insert_non_full
7 9 None
6
leaf -1
    4 None None
         1 2 3
         6 7 9


insert 8
root not full
insert_non_full
4 None None
8
children 1
done split
    4 7 None
         1 2

AttributeError: 'NoneType' object has no attribute 'full'